**Status**
* Starter notebook for Parsing: Finance Notice



In [ ]:
from bs4 import BeautifulSoup as Soup
from IPython.display import display, HTML
import pandas as pd
import re
import pprint
import json

In [ ]:
fn = 'procPublicationRequest Oct-Dec 2014 (Updated) - Sheet1-2.csv'
rows = pd.read_csv(fn, header=0)

In [ ]:
agency = rows['AgencyName'] == "Finance"
description = rows['TypeOfNoticeDescription'] == "Notice"
# agency = rows['AgencyName'] == "Fire Department"
# description = rows['TypeOfNoticeDescription'] == "Notice"
target_rows = rows[agency & description]


In [ ]:
def scrape(row):
    output = {}
    if not isinstance(row.AdditionalDescription, str):
        output = { 'error' : 'source is not a string: {}'.format(row.AdditionalDescription) }
    else:
        display(HTML(row.AdditionalDescription))
    display(HTML('<hr/>'))

    row['output'] = json.dumps(output)
    return row

In [ ]:
processed_rows = target_rows.apply(scrape,1)

In [ ]:
errors = []
cols = ['RequestID', 'output', 'AdditionalDescription', 'StartDate', 'EndDate', 'DueDate']
for rec in processed_rows[cols].values:
    id, output, desc, dtStart, dtEnd, dtDue = rec
    output = json.loads(output)
    if output.get('error', None):
        errors.append({'RequestID' : id, 'error': output, 'desc' : desc, 
                       'StartDate' : dtStart, 'EndDate' : dtEnd, 'DueDate' :  dtDue
                      })
        continue

In [ ]:
err_summary = '''<h1>{} Errors parsing Notice::Finance</h1>'''.format(len(errors))
display(HTML(err_summary))
for error in errors:
    pprint.pprint(error)
    print('\n\n')
    desc = error['desc']
    if isinstance(desc, str):
        display(HTML(error['desc']))
    display(HTML('<hr/>'))
